# Classification

In [ ]:
import mne
import numpy as np 
import matplotlib.pyplot as plt
import torch
import os
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from torch.utils.data import DataLoader, Dataset, TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

In [ ]:
main_folder = './EEGData/MNE-eegbci-data/files/eegmmidb/1.0.0'
subdirectories = [f.path for f in os.scandir(main_folder) if f.is_dir()]
opened_files = []
closed_files = []

for subdirectory in subdirectories:
    files = os.listdir(subdirectory)
    if len(files) > 0:
        for file in files:
            if file[-6:] == '01.edf':
                # This is data for eyes opened
                eyes_opened = os.path.join(subdirectory, file)
                opened_files.append(eyes_opened)
            if file[-6:] == '02.edf':
                # This is data for eyes closed
                eyes_closed = os.path.join(subdirectory, file)
                closed_files.append(eyes_closed)
    else:
        print(f"No files found in {subdirectory}")

In [ ]:
large_open_data_raw = []
large_closed_data_row = []

for data in opened_files:
    large_open_data_raw.append(mne.io.read_raw_edf(data, preload=True, verbose=False).get_data(verbose=False))

for data in closed_files:
    large_closed_data_row.append(mne.io.read_raw_edf(data, preload=True, verbose=False).get_data(verbose=False))

print(large_open_data_raw.__len__())
print(large_closed_data_row.__len__())

## Plotting

In [ ]:
def single_channel_plot(data, channel, title):
    plt.figure(figsize=(20, 3))
    plt.plot(data[channel, :])
    plt.title(title)
    plt.show()

single_channel_plot(large_open_data_raw[0], 0, "Channel 0 for eyes opened")
single_channel_plot(large_closed_data_row[0], 0, "Channel 0 for eyes closed")

In [ ]:
def plot_all_channels(data, title):
    plt.figure(figsize=(20, 10))
    plt.plot(data.T)
    plt.title(title)
    plt.show()

plot_all_channels(large_open_data_raw[0], "All channels for eyes opened")
plot_all_channels(large_closed_data_row[0], "All channels for eyes closed")

In [ ]:
def plot_all_channels_subplots(data):
    plt.figure(figsize=(20, 10))
    for i in range(64):
        plt.subplot(8, 8, i+1)
        plt.plot(data[i, :])
        plt.title(f"Channel {i}")
    plt.show() 

plot_all_channels_subplots(large_open_data_raw[0])
plot_all_channels_subplots(large_closed_data_row[0])

## Pre-Processing

In [ ]:
large_closed_data = []
large_open_data = []

data1 = mne.io.read_raw_edf(data, preload=True, verbose=False)

# Define the frequency range for the filter
low_freq = 0.1  # Low-pass frequency in Hz
high_freq = 35 # High-pass frequency in Hz

for data in opened_files[:10]:
    data1 = mne.io.read_raw_edf(data, preload=True, verbose=False)
    data1 = mne.filter.filter_data(data1.get_data(), sfreq=data1.info['sfreq'], l_freq=low_freq, h_freq=high_freq, fir_design="firwin", verbose=False)
    eeg_data1 = data1
    large_open_data.append(eeg_data1)

for data in closed_files[:10]:
    data1 = mne.io.read_raw_edf(data, preload=True, verbose=False)
    data1 = mne.filter.filter_data(data1.get_data(), sfreq=data1.info['sfreq'], l_freq=low_freq, h_freq=high_freq, fir_design="firwin", verbose=False)
    eeg_data1 = data1
    large_closed_data.append(eeg_data1)

plot_all_channels(large_open_data[0], "All channels for eyes opened")
plot_all_channels(large_closed_data[0], "All channels for eyes closed")

In [ ]:
plot_all_channels_subplots(large_open_data_raw[0])
plot_all_channels_subplots(large_open_data[0])

plot_all_channels_subplots(large_closed_data_row[0])
plot_all_channels_subplots(large_closed_data[0])

## PyTorch Set Up

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Single Channel Classification

### Dataset

In [ ]:
channel = 0
data = np.concatenate((np.array(large_closed_data)[:, channel, :], np.array(large_open_data)[:, channel, :]), axis=0)
labels = np.concatenate((np.zeros(len(np.array(large_closed_data)[:, channel, :])), np.ones(len(np.array(large_open_data)[:, channel, :]))), axis=0)
data = torch.Tensor(data).to(device)
labels = torch.Tensor(labels).to(device)
labels = labels.long()

random_indices = np.arange(len(data))
np.random.shuffle(random_indices)
data = data[random_indices]
labels = labels[random_indices]

print(data.shape)
print(labels.shape)

test_size = 0.2 
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=test_size, random_state=42)

train_data = torch.Tensor(train_data)
train_labels = torch.Tensor(train_labels)
test_data = torch.Tensor(test_data)
test_labels = torch.Tensor(test_labels)

batch_size = 32

train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class FNNClassifierSingle(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FNNClassifierSingle, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_size = 9760
hidden_size = 128
num_classes = 2

In [ ]:
fnn_model = FNNClassifierSingle(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fnn_model.parameters(), lr=0.001)

num_epochs = 2000
losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader: 
        optimizer.zero_grad()
        outputs = fnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {epoch_loss:.4f}')
    
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader: 
        outputs = fnn_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test dataset: {100 * correct / total:.2f}%')

## All Channel Classification

In [ ]:
data = np.concatenate((np.array(large_closed_data), np.array(large_open_data)), axis=0)
labels = np.concatenate((np.zeros(len(np.array(large_closed_data))), np.ones(len(np.array(large_open_data)))), axis=0)
data = torch.Tensor(data)
labels = torch.Tensor(labels)
labels = labels.long()

random_indices = np.arange(len(data))
np.random.shuffle(random_indices)
data = data[random_indices]
labels = labels[random_indices]

print(data.shape)
print(labels.shape)

test_size = 0.2 
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=test_size, random_state=42)

train_data = torch.Tensor(train_data)
train_labels = torch.Tensor(train_labels)
test_data = torch.Tensor(test_data)
test_labels = torch.Tensor(test_labels)

batch_size = 32

train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class FNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FNNClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_size = 9760  * 64
hidden_size = 128
num_classes = 2

In [ ]:
fnn_model = FNNClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fnn_model.parameters(), lr=0.001)

num_epochs = 2000
losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader: 
        optimizer.zero_grad()
        outputs = fnn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {epoch_loss:.4f}')
    
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader: 
        outputs = fnn_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test dataset: {100 * correct / total:.2f}%')